Instalações de Bibliotecas e Dependencias

In [ ]:
%pip install perceval
%pip install perceval python-gitlab
%pip install gitpython
%pip install PyGithub
%pip install --upgrade perceval

Clone do Repositorio

In [ ]:
!git clone https://github.com/XRPLF/rippled.git

Extração dos Commits do Repositorio Clonado

In [ ]:
!perceval git rippled --branches develop -o commits.json

Extração dos Pulls Requests Diretamente do Repositorio Remoto

In [ ]:
!perceval github XRPLF rippled --category pull_request -o pulls.json --api-token

Extração das Issues Diretamente do Repositorio Remoto

In [ ]:
!perceval github XRPLF rippled --category issue -o issues.json --sleep-for-rate --api-token 

Codigo para corrigir os arquivos .json retornados pelo Perceval

In [33]:
# Função para adicionar vírgulas após fechamentos de chaves se necessário
def corrigirJsonVirgulas(inputFile, outputFile):
    correctedLines = []
    with open(inputFile, "r") as file:
        lines = file.readlines()

        # Adiciona o "[" na primeira linha
        correctedLines.append("[\n")

        for i in range(len(lines)):
            line = lines[i].strip()

            # Verifica se a linha termina com "}" e se a próxima linha não começa com "]" ou é o fim do arquivo
            if line.endswith("}") and i < len(lines) - 1:
                nextLine = lines[i + 1].strip()

                # Se a próxima linha não for "]" ou o fim, adiciona uma vírgula
                if not (nextLine.startswith("]") or nextLine.startswith("}") or nextLine == ""):
                    correctedLines.append(line + ",\n")
                else:
                    correctedLines.append(line + "\n")
            else:
                correctedLines.append(lines[i])

        # Adiciona o "]" na última linha
        correctedLines.append("]\n")

    # Escreve o arquivo corrigido
    with open(outputFile, "w") as outputFile:
        outputFile.writelines(correctedLines)

# Chama a função para corrigir o arquivo
corrigirJsonVirgulas("commits.json", "commits_correct.json")
corrigirJsonVirgulas("pulls.json", "pulls_correct.json")
corrigirJsonVirgulas("issues.json", "issues_corrected.json")

Código para pegar o Top-5 com mais commits

In [12]:
import json
from collections import Counter

with open('commits_corrected.json') as f:
    commits = json.load(f)

authorCounter = Counter()

for commit in commits:
    author = commit['data']['Author']
    authorCounter[author] += 1

top5Authors = authorCounter.most_common(5)

for author, count in top5Authors:
    print(f"Autor: {author}, Commits: {count}")


Autor: JoelKatz <DavidJoelSchwartz@GMail.com>, Commits: 4424
Autor: Vinnie Falco <vinnie.falco@gmail.com>, Commits: 2719
Autor: Arthur Britto <ahbritto@gmail.com>, Commits: 1744
Autor: Nik Bougalis <nikb@bougalis.net>, Commits: 704
Autor: seelabs <scott.determan@yahoo.com>, Commits: 344


Código para pegar o Top-5 arquivos modificados

In [13]:
with open('commits_corrected.json') as f:
    commits = json.load(f)

arquivosCounter = Counter()

for commit in commits:
    if 'Merge' not in commit['data']:
        arquivos = commit['data']['files']
        for arquivo in arquivos:
            if arquivo['action'] == "M":
                arquivo = arquivo['file']
                arquivosCounter[arquivo] += 1

top5Arquivos = arquivosCounter.most_common(5)

for arquivo, count in top5Arquivos:
    print(f"Arquivo: {arquivo}, Modificações: {count}")


Arquivo: src/ripple/protocol/impl/BuildInfo.cpp, Modificações: 485
Arquivo: src/cpp/ripple/RPCHandler.cpp, Modificações: 319
Arquivo: Builds/VisualStudio2013/RippleD.vcxproj, Modificações: 317
Arquivo: Builds/VisualStudio2013/RippleD.vcxproj.filters, Modificações: 313
Arquivo: src/ripple/app/misc/NetworkOPs.cpp, Modificações: 301


Código para pegar a media de arquivos por commit

In [34]:
with open('commits_correct.json') as f:
    commits = json.load(f)

commitsCounter = 0
arquivosCounter = 0

for commit in commits:
    commitsCounter += 1
    arquivos = commit['data']['files']
    for arquivo in arquivos:
        arquivosCounter += 1

print(f"Número de commits: {commitsCounter}")
print(f"Número de arquivos: {arquivosCounter}")
print(f"Média de Arquivos por Commits: {round(arquivosCounter / commitsCounter)}")


Número de commits: 13158
Número de arquivos: 126907
Média de Arquivos por Commits: 10


Código para pegar a media de dias para fechar uma issue.

In [36]:
from datetime import datetime

with open('issues_correct.json') as f:
    issues = json.load(f)

issuesCounter = 0
count = 0

for issue in issues:
    if 'pull_request' not in issue['data'] and issue['data']['state'] == 'closed':
        issuesCounter += 1
        dataCriacao = datetime.strptime(issue['data']['created_at'], "%Y-%m-%dT%H:%M:%SZ")
        dataClosed = datetime.strptime(issue['data']['closed_at'], "%Y-%m-%dT%H:%M:%SZ")

        tempo = dataClosed - dataCriacao
        count += tempo.days  

print(f"Média de dias para fechar uma issue: {round(count // issuesCounter)}.")


Média de dias para fechar uma issue: 378.


Código para pegar o percentual de issues fechadas associadas a commits.

In [44]:
with open('issues_correct.json') as f:
    issues = json.load(f)

with open('commits_correct.json') as f:
    commits = json.load(f)

closedIssues = [issue for issue in issues if issue['data']['state'] == 'closed']

commitsWithIssues = set()
for commit in commits:
    if 'message' in commit['data']:
        message = commit['data']['message']
        for issue in closedIssues:
            issueNumber = f"#{issue['data']['number']}"
            if issueNumber in message:
                commitsWithIssues.add(issue['data']['number'])

totalClosedIssues = len(closedIssues)
closedIssuesWithCommits = len(commitsWithIssues)

if totalClosedIssues > 0:
    percentual = (closedIssuesWithCommits / totalClosedIssues) * 100
    print(f"Percentual de issues fechadas associadas a commits: {percentual:.2f}%")


Percentual de issues fechadas associadas a commits: 27.74%


Código implementando o algoritmo SZZ simplificado.

In [88]:
import json
from datetime import datetime

with open('issues_correct.json') as f:
    issues = json.load(f)

with open('commits_correct.json') as f:
    commits = json.load(f)

issuesCounter = 0
count = 0
datas = []
criacoes = []
issuesNumbers = []

for issue in issues:
    labels = issue['data']['labels']
    for label in labels:
        if label['name'] == 'Bug':
            if 'pull_request' not in issue['data'] and issue['data']['state'] == 'closed':   
                issuesCounter += 1
                dataCriacao = datetime.strptime(issue['data']['created_at'], "%Y-%m-%dT%H:%M:%SZ")
                dataClosed = datetime.strptime(issue['data']['closed_at'], "%Y-%m-%dT%H:%M:%SZ")

                datas.append(dataClosed - dataCriacao)
                criacoes.append(dataCriacao)
                issuesNumbers.append(issue['data']['number'])

if len(datas) > 0:
    idxMenorDiferenca = datas.index(min(datas))
    dataMenorTempo = criacoes[idxMenorDiferenca]
    numeroIssueEscolhida = f"#{issuesNumbers[idxMenorDiferenca]}"

commitsWithIssues = set()
for commit in commits:
    if numeroIssueEscolhida in commit['data']['message']:
        arquivos = commit['data']['files']
        for arquivo in arquivos:
            if arquivo['action'] == 'M':
                arquivoModificado = arquivo['file']

encontrou = False
for commit in reversed(commits):
    dataCommit = datetime.strptime(commit['data']['CommitDate'], "%a %b %d %H:%M:%S %Y %z")
    dataCommit = dataCommit.strftime("%Y-%m-%d %H:%M:%S")  
    dataCommit = datetime.strptime(dataCommit, "%Y-%m-%d %H:%M:%S")
    if dataCommit < dataMenorTempo:        
        arquivos = commit['data']['files']
        for arquivo in arquivos:    
            if arquivoModificado == arquivo['file']:
                encontrou = True
                print(f"O commit que introduziu o Bug foi: {commit['data']['commit']}")
    if encontrou:
        break


O commit que introduzio o Bug foi: f74b469e6832db56fc286bf49bf1e5bdb01b93b1


Código para pegar a media de comentarios por issue.

In [35]:
import json

with open('issues_correct.json') as f:
    issues = json.load(f)

issuesCounter = 0
comentariosCounter = 0 

for issue in issues:
    if 'pull_request' not in issue['data']:
        issuesCounter += 1
        comentariosCounter += issue['data']['comments']

print(f"Número de issues: {issuesCounter}")
print(f"Número de comentários: {comentariosCounter}")
print(f"Média de comentários por issue: {round(comentariosCounter / issuesCounter)}")


Número de issues: 1391
Número de comentarios: 5533
Media de comentarios por issue: 4
